# 목차:

## * [1.워드클라우드](#1.워드클라우드)
## * [2.지도 시각화](#2.지도시각화)
## * [3.광진구](#3.광진구)
## * [4.서초구](#4.서초구)
## * [5.마포구](#5.마포구)
## * [6.영등포구](#6.영등포구)
## * [7.송파구](#7.송파구)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
warnings.filterwarnings(action='ignore')

plt.rcParams['font.family'] = 'NanumGothic'

#시각화 색상 통일

col=['#6B4E24','#EECA98','#EBAA4F','#6B5B45','#B8853E','#AB891A','#6B5610','#EBBC23','#F7B50F']

# 1. 워드클라우드 <a class="anchor" id="1.워드클라우드"></a>
---
### 9기 조의현

In [ ]:
# open-api로 블로그 json 파일 가져오기

import urllib.request
import requests

Id = str(input('아이디: '))
password = str(input('비번: '))
text = input('블로그 검색어 입력: ')
txt = urllib.parse.quote(text)
num =  input("출력결과 개수: ")

open_api = 'https://openapi.naver.com/v1/search/blog?query=' + txt + "&display=" + num
header_params = {'X-Naver-Client-Id' : Id, 'X-Naver-Client-Secret' : password}
res = requests.get(open_api, headers = header_params)

data = res.json()

In [ ]:
# json파일에서 블로그 포스팅 링크 가져오기

links = []

for _ in range(len(data['items'])):
    links.append(data['items'][_]['link'])

In [ ]:
# 크롬드라이버 가상환경에 설치

import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(2)

In [ ]:
# 블로그 포스트 본문 크롤링

from tqdm import tqdm
from selenium.webdriver.common.by import By

blog_posts = []

for _ in tqdm(links):
    driver.get(_)
    time.sleep(1)
    
    driver.switch_to.frame('mainFrame')
    
    try:
        post = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text
        blog_posts.append(post.replace('\n', ' '))
    except:
        post = driver.find_element(By.CSS_SELECTOR, 'div#content-area').text
        blog_posts.append(post.replace('\n', ' '))
        
driver.quit()

In [ ]:
# df에 데이터 할당 후 csv파일로 저장

import pandas as pd

df = pd.DataFrame({'blog' : blog_posts})
df.to_csv('naver_blog_review.csv', index = False)

**2. 블로그 포스팅 내용으로 워드클라우드 제작** <br>
출처 : 2021-fall-basicsession 텍스트마이닝 섹션

In [ ]:
# csv파일 오픈 후 텍스트 파일 불러오기

import re
import konlpy
import pandas as pd

with open(r'./naver_blog_review', encoding='utf-8') as f:
    text = f.readlines()

In [ ]:
# 텍스트 전처리

text_strip = list([i.strip() for i in text if i != '\n'])
text_join = ' '.join(text_strip)
filtered_content = re.sub(r'[^\d\s\w]', ' ', text_join)

In [ ]:
# 한국어 텍스트 형태소분석기 komoran 정의

komoran = konlpy.tag.Komoran()
komoran_pos = komoran.pos(filtered_content)

komoran.morphs(filtered_content)

komoran_nouns = komoran.nouns(filtered_content)

In [ ]:
# 불용어처리 후 토큰화

stop_words = []

def tokenizer(text):
    nouns = komoran.nouns(text)
    go_words = [noun for noun in nouns if noun not in stop_words]
    return [go_word for go_word in go_words if len(go_word)>1]

filtered_double = tokenizer(filtered_content)

In [ ]:
# 단어 갯수 count

from collections import Counter
c = Counter(filtered_double)
frequent = c.most_common(10)

In [ ]:
# 워드클라우드 제작 -> 벚꽃 wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from os import path
from wordcloud import ImageColorGenerator

FONT_PATH = "C:/windows/fonts/malgun.ttf"

wordcloud1 = WordCloud(
        font_path=FONT_PATH,
        background_color = "black",
        random_state = 1,
        color_func = ImageColorGenerator(img),        
        mask = img
)
wordcloud1.generate_from_frequencies(c)
wordcloud1.to_image()

In [ ]:
# 워드클라우드 제작 -> 미세먼지 wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from os import path
from wordcloud import ImageColorGenerator
import numpy as np

FONT_PATH = "C:/windows/fonts/malgun.ttf"

def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return("hsl(40, 70%%, %d%%)" % np.random.randint(45,55))

img = plt.imread("blossom.jpg")

wordcloud1 = WordCloud(
        font_path=FONT_PATH,
        background_color = "black",
        random_state = 1,       
        mask = img
)
wordcloud1.generate_from_frequencies(c)
wordcloud1.recolor(color_func = color_func)
wordcloud1.to_image()

# 2. 지도 시각화 <a class="anchor" id="2.지도시각화"></a>
---
### 9기 김서진

### 0. 데이터 준비

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json

In [ ]:
year_cont_df = pd.read_csv('/Users/gimseojin/Desktop/2023-1/DSL/EDA_project/EDA_data/서울시 년도별 평균 대기오염도 정보.csv',
                           skiprows=0, encoding = 'EUC-KR')
year_cont_df.head()

In [ ]:
# 편의를 위해 열 이름 변경
year_cont_df.rename(columns={"측정년도":"Year", "측정소명":"Where","이산화질소농도(ppm)":"NO2",
                             "오존농도(ppm)":"O3","일산화탄소농도(ppm)":"CO",
                            "아황산가스(ppm)":"SO2", "미세먼지(㎍/㎥)":"Dust",
                             "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)
year_cont_df

In [ ]:
# 결측치 보간
mean_no2 = year_cont_df['NO2'].mean()
mean_o3 = year_cont_df['O3'].mean()
mean_co = year_cont_df['CO'].mean()
mean_so2 = year_cont_df['SO2'].mean()
year_cont_df['NO2'].fillna(mean_no2, inplace=True)
year_cont_df['O3'].fillna(mean_o3, inplace=True)
year_cont_df['CO'].fillna(mean_co, inplace=True)
year_cont_df['SO2'].fillna(mean_so2, inplace=True)
#year_cont_df['Dust'].interpolate(method = 'linear', inplace = True)
#year_cont_df['F_dust'].interpolate(method = 'linear', inplace = True)
year_cont_df.isnull().sum()

In [ ]:
# 지역별로 데이터 그룹화
group_2 = year_cont_df.groupby('Where')

In [ ]:
geo_path = 'EDA_data/HangJeongDong_ver20220701.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8')) 

In [ ]:
mean = group_2.mean()
mean.head()
mean.drop('Year', axis = 1, inplace = True)

In [ ]:
map3 = folium.Map(location=[37.5502, 126.982],   # 지도의 중심 위치(위도, 경도)
				 zoom_start=11,                 # 0~18 단계
                 tiles='cartodbpositron') # Stamen Toner, Stamen Terrain, ...

bins_NO2 = list(mean['NO2'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))

# 지도에 영역 표시
folium.Choropleth(geo_data = geo_str,
               data = mean['NO2'],
               columns = [mean.index, mean['NO2']],
               fill_color = 'YlOrBr', #PuRd, YlGnBu
               key_on = 'feature.properties.sggnm',
               highlight = True, 
               legend = '지역구별 이산화질소 연간 평균 농도',
               bins=bins_NO2).add_to(map3)
map3

In [ ]:
map5 = folium.Map(location=[37.5502, 126.982],   # 지도의 중심 위치(위도, 경도)
				 zoom_start=11,                 # 0~18 단계
                 tiles='cartodbpositron') # Stamen Toner, Stamen Terrain, ...

bins_O3 = list(mean['O3'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))

# 지도에 영역 표시
folium.Choropleth(geo_data = geo_str,
               data = mean['O3'],
               columns = [mean.index, mean['O3']],
               fill_color = 'YlOrBr', #PuRd, YlGnBu
               key_on = 'feature.properties.sggnm',
               highlight = True, 
               legend = '지역구별 오존 연간 평균 농도',
               bins=bins_O3).add_to(map5)
map5.save('지역구별 오존 연간 평균 농도.html')
map5

In [ ]:
map6 = folium.Map(location=[37.5502, 126.982],   # 지도의 중심 위치(위도, 경도)
				 zoom_start=11,                 # 0~18 단계
                 tiles='cartodbpositron') # Stamen Toner, Stamen Terrain, ...

bins_O3 = list(mean['Dust'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))

# 지도에 영역 표시
folium.Choropleth(geo_data = geo_str,
               data = mean['Dust'],
               columns = [mean.index, mean['Dust']],
               fill_color = 'YlOrBr', #PuRd, YlGnBu
               key_on = 'feature.properties.sggnm',
               highlight = True, 
               legend = '지역구별 미세먼지 연간 평균 농도',
               bins=bins_O3).add_to(map6)

map6.save('지역구별 미세먼지 연간 평균 농도.html')
map6

In [ ]:
map7 = folium.Map(location=[37.5502, 126.982],   # 지도의 중심 위치(위도, 경도)
				 zoom_start=11,                 # 0~18 단계
                 tiles='cartodbpositron') # Stamen Toner, Stamen Terrain, ...

bins_Fdust = list(mean['F_dust'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))

# 지도에 영역 표시
folium.Choropleth(geo_data = geo_str,
               data = mean['F_dust'],
               columns = [mean.index, mean['F_dust']],
               fill_color = 'YlOrBr', #PuRd, YlGnBu
               key_on = 'feature.properties.sggnm',
               highlight = True, 
               legend = '지역구별 초미세먼지 연간 평균 농도',
               bins=bins_Fdust).add_to(map7)

map7.save('지역구별 연간 초미세먼지 평균 농도.html')
map7

# 3. 광진구 <a class="anchor" id="3.광진구"></a>
---
### 9기 박서연

## 1. 광진구 연도별 분석

In [ ]:
### calling packages

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

import time
from datetime import datetime

### 0. 통일성을 위한 공통 작업

In [ ]:
### matplotlib, seaborn 한글 깨짐 현상 해결

from matplotlib import font_manager, rc # matplotlib, seaborn에서 한글 깨짐 현상을 해결하기 위함

font_path = "C:/WINDOWS/FONTS/MALGUNSL.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [ ]:
### 색상 통일

col=['#6B4E24','#EECA98','#EBAA4F','#6B5B45','#B8853E','#AB891A','#6B5610','#EBBC23','#F7B50F']

### 1. 연도별 평균 데이터를 이용해 전체적인 추이를 보자!

In [ ]:
### '서울시 년도별 평균 대기오염도 정보.csv'를 ATM_yearly_df라는 데이터프레임 형태로 불러오기

ATM_yearly_df = pd.read_csv('C:/Users/izzle/Downloads/서울시 년도별 평균 대기오염도 정보.csv', encoding='cp949')
ATM_yearly_df.head(10)

In [ ]:
### 광진구 자료만 추출하기

gwangjin_yearly = ATM_yearly_df[ATM_yearly_df['측정소명'] == '광진구']
gwangjin_yearly.head(10)

In [ ]:
### 결측치

gwangjin_yearly.isnull().sum()

# 연도별 데이터의 경우 미세먼지, 초미세먼지에만 결측치가 존재
# 미세먼지, 초미세먼지를 제외한 다른 대기오염물질의 경우 측정이 1987년부터 시작되었는데,
# 미세먼지는 2000년부터, 초미세먼지는 2012년부터 측정 데이터가 존재하여 발생하는 결측치이므로 보간할 필요가 없다고 판단됨.

In [ ]:
### 연도 범위 파악

print(gwangjin_yearly.max()['측정년도'])
print(gwangjin_yearly.min()['측정년도'])

In [ ]:
### 초미세먼지 범위 파악

pd.DataFrame(gwangjin_yearly.dropna(axis=0, inplace=False))

# 서울시 년도별 평균 대기오염도 정보 파일에서 초미세먼지는 2012~2023 데이터가 존재

In [ ]:
### 미세먼지 범위 파악

gwangjin_noPM25 = gwangjin_yearly.drop(['초미세먼지(㎍/㎥)'], axis=1, inplace=False)
gwangjin_noPM25.dropna(axis=0, inplace=False)

# 서울시 년도별 평균 대기오염도 정보 파일에서 미세먼지는 2000~2023 데이터가 존재

In [ ]:
### 광진구 이산화탄소, 오존 농도 시계열 그래프(1987~2023)

gwangjin_yearly.plot(x='측정년도', y=['이산화질소농도(ppm)', '오존농도(ppm)'], color = col, figsize=(25, 7))
plt.ylabel('농도(ppm)')
plt.xlabel('측정년도')
plt.title('광진구 이산화질소, 오존 농도 시계열 그래프(1987~2023)', fontsize = 15, pad = 20)

# plt.savefig('광진구 이산화질소, 오존 농도 시계열.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 광진구 미세먼지(2000~2023), 초미세먼지(2012~2023) 시계열 그래프

gwangjin_yearly.plot(x='측정년도', y=['미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)'], color = col, figsize=(25, 7))
plt.ylabel('농도(㎍/㎥)')
plt.xlabel('측정년도')
plt.title('광진구 미세먼지(2000~2023), 초미세먼지(2012~2023) 시계열 그래프', fontsize = 15, pad = 20)

# plt.savefig('광진구 미세먼지, 초미세먼지 시계열.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

## 2. 광진구 월별 분석

### 월별 평균 데이터를 통해 월별 경향성을 살펴보자!

##### 전체 기간 월별 시계열 그래프

In [ ]:
### '서울시 월별 평균 대기오염도 정보.csv'를 ATM_monthly_df라는 데이터프레임 형태로 불러오기

ATM_monthly_df = pd.read_csv('C:/Users/izzle/Downloads/서울시 월별 평균 대기오염도 정보.csv', encoding='cp949')
ATM_monthly_df.head(10)

In [ ]:
### 광진구 자료만 추출하기

gwangjin_monthly = ATM_monthly_df[ATM_monthly_df['측정소명'] == '광진구']
gwangjin_monthly

In [ ]:
### 결측치 확인

gwangjin_monthly.isnull().sum()

In [ ]:
### 결측치 보간
# 미세먼지, 초미세먼지 결측치가 많은 것은 측정 시작 시점의 차이 때문이라서 우선은 이산화질소, 오존에 대해서만 평균값을 이용해 보간

mean_NO2 = gwangjin_monthly['이산화질소농도(ppm)'].mean()
mean_O3 = gwangjin_monthly['오존농도(ppm)'].mean()

gwangjin_monthly['이산화질소농도(ppm)'].fillna(mean_NO2, inplace = True)
gwangjin_monthly['오존농도(ppm)'].fillna(mean_O3, inplace = True)

In [ ]:
### datetime 적용하기

gwangjin_monthly['day'] = 1
gwangjin_monthly['year'] = gwangjin_monthly['측정월']//100
gwangjin_monthly['month'] = gwangjin_monthly['측정월']%100
gwangjin_monthly.head(10)

In [ ]:
### datetime 적용하기

gwangjin_monthly['date'] = pd.to_datetime(gwangjin_monthly[['year', 'month', 'day']])
gwangjin_monthly.head(10)

In [ ]:
gwangjin_18_22 = gwangjin_monthly.loc[9:2886]
gwangjin_18_22.head()

##### 2018~2022 최근 5개년 월별 box plot

In [ ]:
### 이산화질소 농도 box plot

plt.figure(figsize = (10, 6))
plt.boxplot([gwangjin_18_22[gwangjin_18_22['month'] == 1]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 2]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 3]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 4]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 5]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 6]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 7]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 8]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 9]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 10]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 11]['이산화질소농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 12]['이산화질소농도(ppm)']], sym = 'b*')

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.title('광진구 최근 5개년 월별 이산화질소 농도 box plot', fontsize = 15, pad = 20)
plt.xlabel('월')
plt.ylabel('농도(ppm)')

# plt.savefig('광진구 최근 5개년 월별 이산화질소 농도 box plot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 오존 농도 box plot

plt.figure(figsize = (10, 6))
plt.boxplot([gwangjin_18_22[gwangjin_18_22['month'] == 1]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 2]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 3]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 4]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 5]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 6]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 7]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 8]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 9]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 10]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 11]['오존농도(ppm)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 12]['오존농도(ppm)']], sym = 'b*')

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.title('광진구 최근 5개년 월별 오존 농도 box plot', fontsize = 15, pad = 20)
plt.xlabel('월')
plt.ylabel('농도(ppm)')

# plt.savefig('광진구 최근 5개년 월별 오존 농도 box plot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 미세먼지 농도 box plot

plt.figure(figsize = (10, 6))
plt.boxplot([gwangjin_18_22[gwangjin_18_22['month'] == 1]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 2]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 3]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 4]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 5]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 6]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 7]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 8]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 9]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 10]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 11]['미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 12]['미세먼지(㎍/㎥)']], sym = 'b*')

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.title('광진구 최근 5개년 월별 미세먼지 농도 box plot', fontsize = 15, pad = 20)
plt.xlabel('월')
plt.ylabel('농도(㎍/㎥)')

# plt.savefig('광진구 최근 5개년 월별 미세먼지 농도 box plot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 초미세먼지 농도 box plot

plt.figure(figsize = (10, 6))
plt.boxplot([gwangjin_18_22[gwangjin_18_22['month'] == 1]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 2]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 3]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 4]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 5]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 6]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 7]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 8]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 9]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 10]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 11]['초미세먼지(㎍/㎥)'],
            gwangjin_18_22[gwangjin_18_22['month'] == 12]['초미세먼지(㎍/㎥)']], sym = 'b*')

plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'])
plt.title('광진구 최근 5개년 월별 초미세먼지 농도 box plot', fontsize = 15, pad = 20)
plt.xlabel('월')
plt.ylabel('농도(㎍/㎥)')

# plt.savefig('광진구 최근 5개년 월별 초미세먼지 농도 box plot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

##### 이산화질소 계절성 검증을 위한 Two sample t-test

H0: 이산화질소의 여름 평균과 겨울 평균은 같다  
H1: 이산화질소의 여름 평균이 겨울 평균보다 작다(단측검정)

In [ ]:
### 이산화질소 농도 결측치가 있는 행 제거

nonna_NO2_monthly = gwangjin_monthly.dropna(subset=['이산화질소농도(ppm)'])
nonna_NO2_monthly.head()

In [ ]:
### 결측치 재확인

nonna_NO2_monthly.isnull().sum()

In [ ]:
### 여름(6~8월) 데이터 추출하기

summer_NO2_monthly = nonna_NO2_monthly[((nonna_NO2_monthly['month'] > 5) & (nonna_NO2_monthly['month'] < 9))]
summer_NO2_monthly.head(10)

In [ ]:
### 겨울(12~2월) 데이터 추출하기

winter_NO2_monthly = nonna_NO2_monthly[((nonna_NO2_monthly['month'] == 12) | (nonna_NO2_monthly['month'] < 3))]
winter_NO2_monthly.head(10)

In [ ]:
### 이산화질소 데이터 추출하기

summer_NO2_monthly = summer_NO2_monthly['이산화질소농도(ppm)']
winter_NO2_monthly = winter_NO2_monthly['이산화질소농도(ppm)']

In [ ]:
### 확률밀도함수

plt.figure(figsize = (10, 6))
sns.kdeplot(data=summer_NO2_monthly, palette = col, shade=True, label='여름(6~8월)')
sns.kdeplot(data=winter_NO2_monthly, palette = col, shade=True, label='겨울(12~2월)')
plt.legend()
plt.title('이산화질소 농도의 여름 vs 겨울 확률밀도함수', fontsize = 15, pad = 20)

# plt.savefig('이산화질소 여름vs겨울 확률밀도함수.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### Two sample t-test 수행

t_stat, p_value = stats.ttest_ind(summer_NO2_monthly, winter_NO2_monthly, equal_var=True, alternative="less")
print("t-statistics : {}, p-value : {}".format(t_stat, p_value))

p-value < 0.05 이므로 귀무가설을 기각할 수 있다.  
이산화질소의 여름 평균이 겨울 평균에 비해 낮다고 말할 수 있다. 즉, 이산화질소에 계절성이 존재한다.

##### 오존 계절성 검증을 위한 Two sample t-test

H0: 오존의 여름 평균과 겨울 평균은 같다  
H1: 오존의 여름 평균이 겨울 평균보다 크다(단측검정)

In [ ]:
### 오존 농도 결측치가 있는 행 제거

nonna_O3_monthly = gwangjin_monthly.dropna(subset=['오존농도(ppm)'])
nonna_O3_monthly.head()

In [ ]:
### 결측치 재확인

nonna_O3_monthly.isnull().sum()

In [ ]:
### 여름(6~8월) 데이터 추출하기

summer_O3_monthly = nonna_O3_monthly[((nonna_O3_monthly['month'] > 5) & (nonna_O3_monthly['month'] < 9))]
summer_O3_monthly.head(10)

In [ ]:
### 겨울(12~2월) 데이터 추출하기

winter_O3_monthly = nonna_O3_monthly[((nonna_O3_monthly['month'] == 12) | (nonna_O3_monthly['month'] < 3))]
winter_O3_monthly.head(10)

In [ ]:
### 오존 데이터 추출하기

summer_O3_monthly = summer_O3_monthly['오존농도(ppm)']
winter_O3_monthly = winter_O3_monthly['오존농도(ppm)']

In [ ]:
### 확률밀도함수

plt.figure(figsize = (10, 6))
sns.kdeplot(data=summer_O3_monthly, palette = col, shade=True, label='여름(6~8월)')
sns.kdeplot(data=winter_O3_monthly, palette = col, shade=True, label='겨울(12~2월)')
plt.legend()
plt.title('오존 농도의 여름 vs 겨울 확률밀도함수', fontsize = 15, pad = 20)

# plt.savefig('오존 여름vs겨울 확률밀도함수.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### Two sample t-test 수행

t_stat, p_value = stats.ttest_ind(summer_O3_monthly, winter_O3_monthly, equal_var=True, alternative="greater")
print("t-statistics : {}, p-value : {}".format(t_stat, p_value))

p-value < 0.05 이므로 귀무가설을 기각할 수 있다.  
오존의 여름 평균이 겨울 평균에 비해 높다고 말할 수 있다. 즉, 이산화질소에 계절성이 존재한다.

## 3. 광진구 요일별 분석

### 기간별 일평균 데이터를 통해 요일 경향성을 살펴보자!

In [ ]:
### '서울시 기간별 일평균 대기환경 정보.csv'를 ATM_daily_df라는 데이터프레임 형태로 불러오기

ATM_daily_df = pd.read_csv('C:/Users/izzle/Downloads/서울시 기간별 일평균 대기환경 정보.csv', encoding='cp949')
ATM_daily_df.head(10)

In [ ]:
### 광진구 자료만 추출하기

gwangjin_daily = ATM_daily_df[ATM_daily_df['측정소명'] == '광진구']
gwangjin_daily.head(10)

In [ ]:
### 데이터 타입

gwangjin_daily.dtypes

In [ ]:
### 결측치 확인

gwangjin_daily.isnull().sum()

In [ ]:
### datetime 적용하기

gwangjin_daily['측정일자'] = gwangjin_daily['측정일자'].astype(str)
gwangjin_daily['측정일자'] = pd.to_datetime(gwangjin_daily['측정일자'])

In [ ]:
### 데이터 타입 재확인

gwangjin_daily.dtypes

In [ ]:
### 요일 알아내기
# 0: 월요일, 1: 화요일, 2: 수요일, 3: 목요일, 4: 금요일, 5: 토요일, 6: 일요일

gwangjin_daily['요일'] = gwangjin_daily['측정일자'].dt.weekday
gwangjin_daily.head()

In [ ]:
### 요일별 평균 알아내기

day_group = gwangjin_daily.groupby('요일')
day_group.mean()

In [ ]:
### 이산화질소 농도에 대해 내림차순

day_group.mean().sort_values(by='이산화질소농도(ppm)', ascending=False, inplace=False)

# 이산화질소 농도는 평균적으로 목요일이 가장 높고, 일요일이 가장 낮다

In [ ]:
### 오존 농도에 대해 내림차순

day_group.mean().sort_values(by='오존(ppm)', ascending=False, inplace=False)

# 오존 농도는 평균적으로 일요일이 가장 높고, 월요일이 가장 낮다

In [ ]:
### 미세먼지 농도에 대해 내림차순

day_group.mean().sort_values(by='미세먼지(㎍/㎥)', ascending=False, inplace=False)

# 미세먼지 농도는 평균적으로 토요일이 가장 높고, 월요일이 가장 낮다

In [ ]:
### 초미세먼지 농도에 대해 내림차순

day_group.mean().sort_values(by='초미세먼지(㎍/㎥)', ascending=False, inplace=False)

# 초미세먼지 농도는 평균적으로 금요일이 가장 높고, 월요일이 가장 낮다

##### 요일별 barplot

In [ ]:
### 이산화질소 농도 요일별 barplot

plt.figure(figsize = (10, 6))
sns.barplot(data = gwangjin_daily, x = '요일', y = '이산화질소농도(ppm)', palette = col, order = [0, 1, 2, 3, 4, 5, 6])
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['월', '화', '수', '목', '금', '토', '일'])
plt.title('광진구 요일별 이산화질소 농도 barplot', fontsize = 15, pad = 20)

# plt.savefig('광진구 이산화질소 농도 요일별 barplot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 오존 농도 요일별 barplot

plt.figure(figsize = (10, 6))
sns.barplot(data = gwangjin_daily, x = '요일', y = '오존(ppm)', palette = col, order = [0, 1, 2, 3, 4, 5, 6])
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['월', '화', '수', '목', '금', '토', '일'])
plt.title('광진구 요일별 오존 농도 barplot', fontsize = 15, pad = 20)

# plt.savefig('광진구 오존 농도 요일별 barplot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 미세먼지 농도 요일별 barplot

plt.figure(figsize = (10, 6))
sns.barplot(data = gwangjin_daily, x = '요일', y = '미세먼지(㎍/㎥)', palette = col, order = [0, 1, 2, 3, 4, 5, 6])
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['월', '화', '수', '목', '금', '토', '일'])
plt.title('광진구 요일별 미세먼지 농도 barplot', fontsize = 15, pad = 20)

# plt.savefig('광진구 미세먼지 농도 요일별 barplot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 초미세먼지 농도 요일별 barplot

plt.figure(figsize = (10, 6))
sns.barplot(data = gwangjin_daily, x = '요일', y = '초미세먼지(㎍/㎥)', palette = col, order = [0, 1, 2, 3, 4, 5, 6])
plt.xticks([0, 1, 2, 3, 4, 5, 6], ['월', '화', '수', '목', '금', '토', '일'])
plt.title('광진구 요일별 초미세먼지 농도 barplot', fontsize = 15, pad = 20)

# plt.savefig('광진구 초미세먼지 농도 요일별 barplot.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

## 4. 광진구 시간별 분석

### 기간별 시간평균 데이터를 통해 일변화 경향성을 살펴보자!

In [ ]:
### '서울시 기간별 시간평균 대기환경 정보.csv'를 ATM_daily_df라는 데이터프레임 형태로 불러오기

ATM_hourly_df = pd.read_csv('C:/Users/izzle/Downloads/서울시 기간별 시간평균 대기환경 정보.csv', encoding='cp949')
ATM_hourly_df.head(10)

In [ ]:
### 광진구 자료만 추출하기

gwangjin_hourly = ATM_hourly_df[ATM_hourly_df['측정소명'] == '광진구']
gwangjin_hourly.head(10)

In [ ]:
### 데이터 타입

gwangjin_hourly.dtypes

In [ ]:
### 결측치 확인

gwangjin_hourly.isnull().sum()

In [ ]:
### datetime 적용하기

gwangjin_hourly['측정일시2'] = pd.to_datetime(gwangjin_hourly['측정일시'], format = '%Y/%m/%d %H:%M')
gwangjin_hourly

In [ ]:
### 시간 컬럼 분리하기

gwangjin_hourly['시간'] = gwangjin_hourly['측정일시2'].dt.time
gwangjin_hourly

In [ ]:
### 데이터 타입 재확인

gwangjin_hourly.dtypes

In [ ]:
### '시간만' 추출

gwangjin_hourly['time'] = gwangjin_hourly.apply(lambda row: int(str(row['시간'])[:2]), axis=1)
gwangjin_hourly

In [ ]:
### 데이터 처리

mean_NO2_by_hour = gwangjin_hourly.groupby('time')['이산화질소농도(ppm)'].mean()
mean_O3_by_hour = gwangjin_hourly.groupby('time')['오존(ppm)'].mean()
mean_PM10_by_hour = gwangjin_hourly.groupby('time')['미세먼지 1시간(㎍/㎥)'].mean()
mean_PM25_by_hour = gwangjin_hourly.groupby('time')['초미세먼지(㎍/㎥)'].mean()

In [ ]:
### 이산화질소, 오존 농도 시간별 평균 그래프

plt.figure(figsize = (10, 6))
plt.plot(mean_NO2_by_hour, color = '#6B4E24', label = '이산화질소농도')
plt.plot(mean_O3_by_hour, color = '#EECA98', label = '오존농도')
plt.xticks(np.arange(0, 24))
plt.xlabel('시간')
plt.ylabel('농도(ppm)')
plt.legend()
plt.title('광진구 이산화질소, 오존 농도 시간별 평균 그래프', fontsize = 15, pad = 20)

# plt.savefig('광진구 이산화질소, 오존 농도 시간별 평균.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

In [ ]:
### 미세먼지, 초미세먼지 농도 시간별 평균 그래프

plt.figure(figsize = (10, 6))
plt.plot(mean_PM10_by_hour, color = '#6B4E24', label = '미세먼지농도')
plt.plot(mean_PM25_by_hour, color = '#EECA98', label = '초미세먼지농도')
plt.xticks(np.arange(0, 24))
plt.xlabel('시간')
plt.ylabel('농도(㎍/㎥)')
plt.legend()
plt.title('광진구 미세먼지, 초미세먼지 농도 시간별 평균 그래프', fontsize = 15, pad = 20)

# plt.savefig('광진구 미세먼지, 초미세먼지 농도 시간별 평균.png', facecolor = 'white', bbox_inches='tight', pad_inches=0.5)

## 5. 벚꽃 개화 시기의 대기오염물질 농도 분석

### 벚꽃 시즌(3월 셋째주~4월 둘째주) 분석

##### 3월 셋째주 주말

In [ ]:
### 3월 셋째주 토요일 분석

blossom_03_03 = pd.merge(blossom_2019_03_03, blossom_2020_03_03, how='left', on='시간')
blossom_03_03['총합'] = blossom_03_03['합']+blossom_03_03['합2']
blossom_03_03.sort_values(by='총합', ascending=True, inplace=False)

# 오전 10시: 총합 96.102

In [ ]:
### 3월 셋째주 일요일 분석

blossom_03_03_2 = pd.merge(blossom_2019_03_03_2, blossom_2020_03_03_2, how='left', on='시간')
blossom_03_03_2['총합'] = blossom_03_03_2['합']+blossom_03_03_2['합2']
blossom_03_03_2.sort_values(by='총합', ascending=True, inplace=False)

# 오후 3시: 총합 43.099

##### 3월 넷째주 주말

In [ ]:
### 3월 넷째주 토요일 분석

blossom_03_04 = pd.merge(blossom_2019_03_04, blossom_2020_03_04, how='left', on='시간')
blossom_03_04['총합'] = blossom_03_04['합']+blossom_03_04['합2']
blossom_03_04.sort_values(by='총합', ascending=True, inplace=False)

# 오후 6시: 총합 53.093

In [ ]:
### 3월 넷째주 일요일 분석

blossom_03_04_2 = pd.merge(blossom_2019_03_04_2, blossom_2020_03_04_2, how='left', on='시간')
blossom_03_04_2['총합'] = blossom_03_04_2['합']+blossom_03_04_2['합2']
blossom_03_04_2.sort_values(by='총합', ascending=True, inplace=False)

# 오후 5시: 총합 80.101

##### 4월 첫째주 주말

In [ ]:
### 4월 첫째주 토요일 분석

blossom_04_01 = pd.merge(blossom_2019_04_01, blossom_2020_04_01, how='left', on='시간')
blossom_04_01['총합'] = blossom_04_01['합']+blossom_04_01['합2']
blossom_04_01.sort_values(by='총합', ascending=True, inplace=False)

# 오후 5시: 총합 134.103

In [ ]:
### 4월 첫째주 일요일 분석

blossom_04_01_2 = pd.merge(blossom_2019_04_01_2, blossom_2020_04_01_2, how='left', on='시간')
blossom_04_01_2['총합'] = blossom_04_01_2['합']+blossom_04_01_2['합2']
blossom_04_01_2.sort_values(by='총합', ascending=True, inplace=False)

# 오후 3시: 총합 110.109

##### 4월 둘째주 주말

In [ ]:
### 4월 둘째주 토요일 분석

blossom_04_02 = pd.merge(blossom_2019_04_02, blossom_2020_04_02, how='left', on='시간')
blossom_04_02['총합'] = blossom_04_02['합']+blossom_04_02['합2']
blossom_04_02.sort_values(by='총합', ascending=True, inplace=False)

# 오후 6시: 총합 98.126

In [ ]:
### 4월 둘째주 일요일 분석

blossom_04_02_2 = pd.merge(blossom_2019_04_02_2, blossom_2020_04_02_2, how='left', on='시간')
blossom_04_02_2['총합'] = blossom_04_02_2['합']+blossom_04_02_2['합2']
blossom_04_02_2.sort_values(by='총합', ascending=True, inplace=False)

# 오후 4시: 총합 89.114

* 보통 피크닉을 주말 낮에 간다는 점을 고려하여 벚꽃 시즌(3월 셋째주~4월 둘째주)의 각 주 주말의 오전 10시~오후 6시 시간대 중   
**대기오염물질 농도 총합이 최소**가 되는 때를 적어 두었습니다.  

* 2019년 1년만의 데이터로만 말하기는 부족할 것 같아서 2020년 데이터를 찾아 2019+2020 총합이 가장 낮은 시간대를 알아보았습니다.  

* 2019년은 3월이 다섯째주까지 있어 최대한 2020년과 날짜를 맞추기 위해, 2019년 3월 넷째주를 셋째주로, 다섯째주를 넷째주로 생각하였습니다.

- 3월 셋째주 주말: 일요일, 오후 3시(2019+2020 총합: 43.099)
- 3월 넷째주 주말: 토요일, 오후 6시(2019+2020 총합 53.093)
- 4월 첫째주 주말: 일요일, 오후 3시(2019+2020 총합 110.109)
- 4월 둘째주 주말: 일요일, 오후 4시(2019+2020 총합 89.114)

# 4. 서초구 <a class="anchor" id="4.서초구"></a>
---
### 9기 임선민

## 서초구 연도별 데이터

- 서초구의 연도별 대기오염물질 농도 추이 관찰
- 서울시 연도별 평균 대기오염도 정보 데이터를 사용함

In [ ]:
# 서초구 연도별 데이터 가져오기
year = pd.read_csv('/content/drive/MyDrive/DSL/EDA/year.csv', encoding='cp949')
s_year = year[year['측정소명'] == '서초구']

#불필요한 column 제거
s_year = s_year[["측정년도", "측정소명", "이산화질소농도(ppm)", "오존농도(ppm)", "미세먼지(㎍/㎥)", "초미세먼지(㎍/㎥)"]]

# 편의를 위해 열 이름 변경
s_year.rename(columns = {"이산화질소농도(ppm)":"NO2", "오존농도(ppm)":"O3", "미세먼지(㎍/㎥)":"Dust", "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)
s_year.head()

print(s_year.shape)
print('-' * 10)
print(s_year.head(11).isnull().sum())

In [ ]:
#결측치 보간 -> 대기변수별 평균값으로 결측치를 채워줌.
mean_Dust = s_year['Dust'].mean()
mean_F_dust = s_year['F_dust'].mean()

s_year['Dust'].fillna(mean_Dust, inplace=True)
s_year['F_dust'].fillna(mean_F_dust, inplace=True)

s_year.isna().sum()

In [ ]:
s_year = s_year[s_year['측정년도'] < 2023]
s_year = s_year[s_year['측정년도'] > 2011]
s_year

### 대기오염물질 분석

In [ ]:
# 이산화질소, 오존 분석
s_year.plot(x='측정년도', y=['NO2', 'O3'], color = col, marker = 'o', figsize=(10, 6))
plt.grid(True)
plt.ylabel('농도(ppm)')
plt.xlabel('측정년도')
plt.title('서초구 이산화질소, 오존 농도 추이 (2012~2022)', fontsize = 15, pad = 15)

In [ ]:
# 미세먼지, 초미세먼지 분석
s_year.plot(x='측정년도', y=['Dust', 'F_dust'], color = col,marker = 'o', figsize=(10, 6))
plt.grid(True)
plt.ylabel('농도(ppm)')
plt.xlabel('측정년도')
plt.title('서초구 미세먼지, 초미세먼지 농도 추이 (2012~2022)', fontsize = 15, pad = 15)

#### 월별 합산 분석

In [ ]:
# 이산화질소 분석
plt.bar(x = s_month['month'], height = s_month['NO2'], color = col)
plt.title('서초구 월별 이산화질소 농도', fontsize=15, pad=15)
plt.xlabel('month')
plt.ylabel('농도(ppm)')

plt.plot(s_month['month'], s_month['NO2'], marker = 'o', color = col[0])

plt.show()

In [ ]:
# 오존 분석
plt.bar(x = s_month['month'], height = s_month['O3'], color = col)
plt.title('서초구 월별 오존 농도', fontsize=15, pad=15)
plt.xlabel('month')
plt.ylabel('농도(ppm)')

plt.plot(s_month['month'], s_month['O3'], marker = 'o', color = col[3])

plt.show()

In [ ]:
# 미세먼지 분석
plt.bar(x = s_month['month'], height = s_month['Dust'], color = col)
plt.title('서초구 월별 미세먼지 농도', fontsize=15, pad=15)
plt.xlabel('month')
plt.ylabel('농도(㎍/㎥)')

plt.plot(s_month['month'], s_month['Dust'], marker = 'o', color = col[0])

plt.show()

In [ ]:
# 초미세먼지 분석
plt.bar(x = s_month['month'], height = s_month['F_dust'], color = col)
plt.title('서초구 월별 초미세먼지 농도', fontsize=15, pad=15)
plt.xlabel('month')
plt.ylabel('농도(㎍/㎥)')

plt.plot(s_month['month'], s_month['F_dust'], marker = 'o', color = col[3])

plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
month_scaled = pd.DataFrame(sc.fit_transform(s_month), index = s_month.index, columns = s_month.columns)
month_scaled

In [ ]:
# 합산 분석
plt.bar(x = month_scaled.index, height = month_scaled['Sum'], color = col)
plt.title('서초구 월별 대기오염물질 농도 (합산)', fontsize=15, pad=15)
#plt.grid()
plt.rcParams['axes.unicode_minus'] = False
plt.xlabel('month')
plt.ylabel('농도')
plt.show()

- 벚꽃개화시기 (3~4월)의 미세먼지, 초미세먼지, 이산화질소의 농도는 다른 월별 데이터와 비교해봤을 때, 상대적으로 매우 높은 수치를 보이며, 오존의 농도 또한 3,4월에 점점 증가하는 추세를 보임.

## 서초구 3&4월 요일별 데이터

- 서초구의 요일별 대기오염물질 농도 추이 관찰
- 서울시 기간별 일평균 대기환경 정보 데이터를 사용함

### 데이터셋 가져오기

In [ ]:
# 서초구 일평균 데이터 가져오기
day = pd.read_csv('/content/drive/MyDrive/DSL/EDA/day.csv', encoding='cp949')
day = day[day['측정소명'] == '서초구']

#불필요한 column 제거
day = day[["측정일자", "측정소명", "이산화질소농도(ppm)", "오존(ppm)", "미세먼지(㎍/㎥)", "초미세먼지(㎍/㎥)"]]

# 편의를 위해 열 이름 변경
day.rename(columns = {"이산화질소농도(ppm)":"NO2", "오존(ppm)":"O3", "미세먼지(㎍/㎥)":"Dust", "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)

print(day.shape)
print('-' * 10)
print(day.isnull().sum())

In [ ]:
#20220301~20220430 3,4월 데이터 추출하기

day = day[(day['측정일자']-20220300) <= 130]
day = day[(day['측정일자']-20220300) >= 1]
day

In [ ]:
day['day'] = [5, 4, 3, 2, 1, 7, 6] * 8 + [5, 4, 3, 2, 1]
day.head()

In [ ]:
# 요일별 합산 데이터 s_day
s_day = day.groupby(['day']).agg({'NO2' : 'mean', 'O3' : 'mean', 'Dust' : 'mean', 'F_dust': 'mean', 'day' : 'mean'})
s_day

In [ ]:
s_day['day'][1] = '월'
s_day['day'][2] = '화'
s_day['day'][3] = '수'
s_day['day'][4] = '목'
s_day['day'][5] = '금'
s_day['day'][6] = '토'
s_day['day'][7] = '일'
s_day

### 대기오염물질 농도 분석

In [ ]:
# 이산화질소 분석
plt.bar(x = s_day['day'], height = s_day['NO2'], color = col)
plt.title('서초구 요일별 이산화질소 농도', fontsize=15, pad=15)
plt.xlabel('day')
plt.ylabel('농도(ppm)')

plt.plot(s_day['day'], s_day['NO2'], marker = 'o', color = col[0])

plt.show()

In [ ]:
# 오존 분석
plt.bar(x = s_day['day'], height = s_day['O3'], color = col)
plt.title('서초구 요일별 오존 농도', fontsize=15, pad=15)
plt.xlabel('day')
plt.ylabel('농도(ppm)')

plt.plot(s_day['day'], s_day['O3'], marker = 'o', color = col[0])

plt.show()

In [ ]:
# 미세먼지 분석
plt.bar(x = s_day['day'], height = s_day['Dust'], color = col)
plt.title('서초구 요일별 미세먼지 농도', fontsize=15, pad=15)
plt.xlabel('day')
plt.ylabel('농도(㎍/㎥)')

plt.plot(s_day['day'], s_day['Dust'], marker = 'o', color = col[0])

plt.show()

In [ ]:
# 초미세먼지 분석
plt.bar(x = s_day['day'], height = s_day['F_dust'], color = col)
plt.title('서초구 요일별 초미세먼지 농도', fontsize=15, pad=15)
plt.xlabel('day')
plt.ylabel('농도(㎍/㎥)')

plt.plot(s_day['day'], s_day['F_dust'], marker = 'o', color = col[0])

plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
day_scaled = pd.DataFrame(sc.fit_transform(s_day), index = s_day.index, columns = s_day.columns)
day_scaled

In [ ]:
day_scaled['Sum'] = day_scaled['Dust']+day_scaled['F_dust']+day_scaled['NO2']+day_scaled['O3']

In [ ]:
hi = pd.DataFrame({'day':[1,2,3,4,5,6,7], 'date' : ['월', '화','수','목','금','토','일',]})
#hi = hi.set_index(keys=['day'], inplace=False, drop=False)
hi

In [ ]:
day_scaled = pd.merge(hi, day_scaled, on="day")

In [ ]:
# 합산 분석
plt.bar(x = day_scaled['date'], height = day_scaled['Sum'], color = col)
plt.title('서초구 요일별 대기오염물질 농도 (합산)', fontsize=15, pad=15)
#plt.grid()
plt.rcParams['axes.unicode_minus'] = False
plt.xlabel('day')
plt.ylabel('농도')
plt.show()

- 미세먼지: 화 > 수 > 월 > 금 > 목 > 일 > 토 -> 주말에 낮은 수치를 보임.
- 초미세먼지: 화 > 월 > 일 > 수 > 목 > 금 > 토 -> 금, 토에 높은 수치를 보이고, 일, 월에 낮은 수피를 보임. 미세먼지와 비슷한 경향성.
- NO2: 일 > 목 > 수 > 화 > 월 > 금 > 토 -> 주중에 높은 수치를 보이고, 주말에 상대적으로 낮아짐.
- O3: 토 > 금 > 목 > 월 > 수 > 일 > 화 -> NO2와 대체로 반대의 경향성을 보임. 토요일에 높은 수치를 보임.

--> 합산: 서초구 3&4월 한강 피크닉은 토요일을 강추!!

## 서초구 3&4월 시간별 데이터

- 서초구의 시간별 대기오염물질 농도 추이 관찰
- 서울시 기간별 시간평균 대기환경 정보 데이터를 사용함

In [ ]:
# 서초구 2019년 일평균 데이터 가져오기
time_2019 = pd.read_csv('/content/drive/MyDrive/DSL/EDA/2019_time.csv')
time_2019 = time_2019[time_2019['측정소명'] == '서초구']

#불필요한 column 제거
time_2019 = time_2019[["측정일시", "측정소명", "이산화질소농도(ppm)", "오존(ppm)", "미세먼지 24시간(㎍/㎥)", "초미세먼지(㎍/㎥)"]]

# 편의를 위해 열 이름 변경
time_2019.rename(columns = {"이산화질소농도(ppm)":"NO2", "오존(ppm)":"O3", "미세먼지 24시간(㎍/㎥)":"Dust", "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)

print(time_2019.shape)
print('-' * 10)
print(time_2019.isnull().sum())
print('-' * 10)
time_2019.head()

In [ ]:
# 서초구 2019년 3&4월 데이터
time_2019 = time_2019[time_2019['측정일시'] >= 201903010000]
time_2019 = time_2019[time_2019['측정일시'] <= 201904302300]
time_2019.sort_values(by=['측정일시'], ascending=True, inplace=True)
time_2019

In [ ]:
time_2019.isna().sum()

In [ ]:
time_2020_3 = pd.read_csv('/content/drive/MyDrive/DSL/EDA/2020_03_time.csv', encoding='cp949')
time_2020_3 = time_2020_3[time_2020_3['측정소명'] == '서초구']

#불필요한 column 제거
time_2020_3 = time_2020_3[["측정일시", "측정소명", "이산화질소농도(ppm)", "오존(ppm)", "미세먼지 24시간(㎍/㎥)", "초미세먼지(㎍/㎥)"]]

# 편의를 위해 열 이름 변경
time_2020_3.rename(columns = {"이산화질소농도(ppm)":"NO2", "오존(ppm)":"O3", "미세먼지 24시간(㎍/㎥)":"Dust", "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)

print(time_2020_3.shape)
print('-' * 10)
print(time_2020_3.isnull().sum())
print('-' * 10)
time_2020_3.head()

In [ ]:
time_2020_4 = pd.read_csv('/content/drive/MyDrive/DSL/EDA/2020_04_time.csv', encoding='cp949')
time_2020_4 = time_2020_4[time_2020_4['측정소명'] == '서초구']

#불필요한 column 제거
time_2020_4 = time_2020_4[["측정일시", "측정소명", "이산화질소농도(ppm)", "오존(ppm)", "미세먼지 24시간(㎍/㎥)", "초미세먼지(㎍/㎥)"]]

# 편의를 위해 열 이름 변경
time_2020_4.rename(columns = {"이산화질소농도(ppm)":"NO2", "오존(ppm)":"O3", "미세먼지 24시간(㎍/㎥)":"Dust", "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)

print(time_2020_4.shape)
print('-' * 10)
print(time_2020_4.isnull().sum())
print('-' * 10)
time_2020_4.head()

In [ ]:
# 서초구 2020년 3&4월 데이터
time_2020 = pd.concat([time_2020_3, time_2020_4])
time_2020

In [ ]:
time_2020.isna().sum()

In [ ]:
# 서초구 2019년 & 2020년 3&4월 데이터 합치기
s_time = pd.concat([time_2019, time_2020])
s_time

In [ ]:
# datetime 적용하기
s_time['측정일시'] = s_time.apply(lambda row: datetime.strptime(str(row['측정일시']), '%Y%m%d%H%M'), axis = 1)
s_time.head(10)

In [ ]:
s_time['월'] = s_time['측정일시'].dt.month
s_time.head(10)

In [ ]:
### 요일 알아내기
# 0: 월요일, 1: 화요일, 2: 수요일, 3: 목요일, 4: 금요일, 5: 토요일, 6: 일요일

s_time['요일'] = s_time['측정일시'].dt.weekday
s_time.head()

In [ ]:
s_time['시간'] = s_time['측정일시'].dt.time
s_time.head(10)

In [ ]:
s_time_sat = s_time[s_time['요일']==5]
s_time_sat

In [ ]:
s_time_sat['time'] = s_time_sat['측정일시'].dt.hour
s_time_sat.head(10)

In [ ]:
time_avg = s_time_sat.groupby(['시간']).agg({'NO2' : 'mean', 'O3' : 'mean', 'Dust' : 'mean', 'F_dust': 'mean', 'time' : 'mean'})
time_avg

In [ ]:
time_avg = time_avg[time_avg['time'] >= 10.0]
time_avg = time_avg[time_avg['time'] <= 21.0]
time_avg

In [ ]:
time_avg.index=[0,1,2,3,4,5,6,7,8,9,10,11]
time_avg

In [ ]:
range = pd.DataFrame({'range':['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C','C','C'], 'time': [10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0]}, )
range
#hi = hi.set_index(keys=['day'], inplace=False, drop=False)
time_avg = pd.merge(time_avg, range)
time_avg

In [ ]:
# A: 10시 ~ 13시 // B: 14시 ~ 17시 // C: 18시 ~ 21시

time_avg = time_avg.groupby(['range']).agg({'NO2' : 'mean', 'O3' : 'mean', 'Dust' : 'mean', 'F_dust': 'mean', 'time' : 'mean'})
time_avg

### 대기오염물질 분석

In [ ]:
# 이산화질소 분석
plt.bar(x = time_avg.index, height = time_avg['NO2'], color = col)
plt.title('서초구 토요일 시간대별 이산화질소 농도', fontsize=15, pad=15)
plt.xlabel('시간대')
plt.ylabel('농도(ppm)')

plt.show()

### 시간별 합산 분석

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
time_scaled = pd.DataFrame(sc.fit_transform(time_avg), index = time_avg.index, columns = time_avg.columns)
time_scaled

In [ ]:
time_scaled['Sum'] = time_scaled['Dust']+time_scaled['F_dust']+time_scaled['NO2']+time_scaled['O3']

In [ ]:
# 합산 분석
plt.bar(x = s_time_sat['time'], height = s_time_sat['sum'], color = col)
plt.title('서초구 토요일 시간별 대기오염물질 농도', fontsize=15, pad=15)
#plt.grid()
plt.rcParams['axes.unicode_minus'] = False
plt.xlabel('time')
plt.ylabel('농도')
plt.show()

# 5. 마포구 <a class="anchor" id="5.마포구"></a>
---
### 9기 조의현

**1. 요일별 마포구 미세먼지 등급 발령 횟수, barplot, piechart** <br>
출처: pandas.DataFrame.apply.html, matplotlib.pyplot.subplots_adjust.html

In [ ]:
# datetime 변수를 요일변수로 할당

week = ['월', '화', '수', '목', '금', '토', '일']

df['요일'] = df.apply(lambda row: week[datetime.date(row['측정일시']).weekday()], axis = 1)

In [ ]:
df_al = df[df['dust'] >= 81]
days = df_al.groupby(['요일']).agg({'dust' : 'count'}).reset_index()
days 

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8,12))

sns.barplot(ax = axes[0], x = days['요일'],
            y = days['dust'], palette = col).set_title('요일별 마포구 미세먼지 등급 발령', size = 20)
axes[0].axis([-0.5, 6.5, 80, 200])
axes[1].pie(labels = days['요일'], x = days['dust'],
            colors = col, textprops = {'fontsize' : 12}, autopct = "%.2f%%")
plt.title('요일별 마포구 미세먼지 등급 발령', size = 16)
plt.subplots_adjust(left = 0.125, bottom = 0.1,  right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.35)
plt.show()

**2. 미세먼지가 가장 심한 날짜시간 Top 10**

**미세먼지가 심한 날 Top 10 중 3월 5일이 2번, 3월 4일이 6번 있는 등 미세먼지가 심한 시간이 몰려서 있는 밀집현상을 보이고 있습니다.**

In [ ]:
# 미세먼지가 가장 심한 날 Top 10

plt.figure(figsize = (12, 8))

dust_srt = df.sort_values('dust', ascending = False)
sns.barplot(x = dust_srt['dust'][:10], y = dust_srt['측정일시'][:10].apply(lambda x : x.strftime('%m월%d일-%H시')), palette = col)
plt.axis([180, 250, 9.5, - 0.5])
plt.show()

**미세먼지 심한 시간대 top 100 중에서 총 13일로 집중되어 있는 집중현상을 발견 가능합니다.**

In [ ]:
# 미세먼지 심한 날이 얼마나 뭉쳐있는지 확인

top_100 = dust_srt.iloc[:100, :]
top_100['day'] = top_100.apply(lambda row: row['측정일시'].strftime('%m%d'), axis = 1)

tp = top_100.groupby(['day']).agg({'dust' : 'count'}).reset_index()
tp = tp.sort_values('dust', ascending = False)

print('# of days in top-100: ', len(tp['day']), '일')
print('-' * 10)

plt.figure(figsize = (12, 9))

sns.barplot(x = tp['day'], y = tp['dust'],
            palette = col)
plt.title('미세먼지 경보 발령 횟수', fontsize = 20)
plt.show()

**3. 미세먼지 경보 발령 시간대**

In [ ]:
df_month = df[df['dust'] >= 81]
df_month['time'] = df_month.apply(lambda row: row['측정일시'].strftime('%H'), axis = 1)

df_t = df_month.groupby(['time']).agg({'dust' : 'count'}).reset_index()
df_t

In [ ]:
plt.figure(figsize = (8, 6))

sns.lineplot(x = df_t['time'], y = df_t['dust'],
            marker = 'o')

plt.show()

# 6. 영등포구 <a class="anchor" id="6.영등포구"></a>
---
### 9기 조세린

## **1. 영등포구 연도별 분석**

In [ ]:
year=pd.read_csv('서울시 년도별 평균 대기오염도 정보_최종.csv',encoding='cp949')

In [ ]:
#전체 데이터 결측치 확인
year.isna().sum()

In [ ]:
#칼럼 이름 변경
year=year.rename(columns={'측정년도':'year','미세먼지(㎍/㎥)':'dust','이산화질소농도(ppm)':'no2','초미세먼지(㎍/㎥)':'fdust',
                          '아황산가스(ppm)':'so2','오존농도(ppm)':'o3','이산화질소농도(ppm)':'no2','일산화탄소농도(ppm)':'co'})

In [ ]:
#결측치 보간 (전체 데이터)

#no2, o3 : 평균으로 보간
year_df1=year[['no2','o3','year']]
year_df1=year_df1.fillna(year_df1.mean())

In [ ]:
#전체 데이터 : 년도별로 groupby
year_df01=year_df1.groupby(by='year').mean()
year_df02=year_df2.groupby(by='year').mean()

In [ ]:
# dust, fdust : 최근 10년치 데이터만 추출 / 최근 10년간은 결측치 없음
year_df2=year[['dust','fdust','year']]
year_df2=year_df2.loc[year_df2['year']>=2013]
year_df2.isna().sum()

In [ ]:
#영등포구 데이터 추출
year_231=year.loc[year['측정소명']=='영등포구']

In [ ]:
#영등포구 데이터 결측치 확인
year_231.isna().sum()

In [ ]:
#필요한 데이터만 가져오기
year_231=year_231[['year','no2','o3','dust','fdust']]

In [ ]:
#측정년도 인덱스화
year_231_df=year_231.set_index('year',drop=False)
year_231_df

In [ ]:
#영등포구 결측치 보간 :no2, o3 /평균으로 제거
year_231_df1=year_231_df[['no2','o3','year']]
year_231_df1=year_231_df1.fillna(year_df1.mean())

In [ ]:
#영등포구 결측치 보간 : 미세먼지, 초미세먼지 / 최근 10년간 데이터만 추출
year_231_df2=year_231_df[['dust','fdust','year']]
year_231_df2=year_231_df2.loc[year_231_df2['year']>=2013]
year_231_df2.isna().sum()

## 데이터셋 요약 
### 1.연도별 분석
#### -사용한 데이터셋 : 서울시 년도별 평균 대기오염도 정보 (1994~2023)
#### -목적 : 서울시 전체 데이터와 비교하여 영등포구의 대기오염도 정도 파악

### >결측치 보간 
#### -이산화질소, 오존 : 평균으로 보간
#### -미세먼지, 초미세먼지 : 결측치가 많아 평균으로 보간하긴 어려움+특정 시기 이전엔 결측치가 많음 => 2013~2023년 10년 간의 데이터만 이용


In [ ]:
#연도별 영등포구 대기환경 변화 : no2, o3
year_231_df1[['no2','o3']].plot(figsize=(10,6),color=col).autoscale(axis='x',tight=True)

In [ ]:
#연도별 영등포구 대기환경 변화 : 미세먼지, 초미세먼지
year_231_df2[['dust','fdust']].plot(figsize=(10,6),color=col).autoscale(axis='x',tight=True)

## **2. 영등포구 월별 분석**

In [ ]:
month=pd.read_csv('서울시 월별 평균 대기오염도 정보_최종.csv',encoding='cp949')

## 데이터셋 요약 
### 2.월별 분석
#### -사용한 데이터셋 : 서울시 월별 평균 대기오염도 정보 (1994~2023)
#### -목적 : 영등포구 내, 3-4월 기간과 1년 평균 비교

### >결측치 보간 
#### -이산화질소, 오존 : 평균으로 보간
#### -미세먼지, 초미세먼지 : 결측치가 많아 평균으로 보간하긴 어려움+특정 시기 이전엔 결측치가 많음 => 2013~2023년 3년 간의 데이터만 이용


In [ ]:
col=['#6B4E24','#EECA98','#EBAA4F','#6B5B45','#B8853E','#AB891A','#6B5610','#EBBC23','#F7B50F']
#이산화질소와 오존 월별 추세
month_231_df01[['no2','o3']].plot(figsize=(10,6),color=col).autoscale(axis='x',tight=True)

#계절성을 나타내는 것 같음 : 계절별 데이터와 비교

In [ ]:
#3,4월 데이터 구분
month_231_df01_0304=month_231_df01.loc[['03','04']]
month_231_df01_12=month_231_df01.loc[['01','02','05','06','07','08','09','10','11','12']]
month_231_df01_0304_no2=month_231_df01_0304['no2']
month_231_df01_0304_o3=month_231_df01_0304['o3']
month_231_df01_12_no2=month_231_df01_12['no2']
month_231_df01_12_o3=month_231_df01_12['o3']

#### no2

In [ ]:
sns.kdeplot(data=month_231_df01_0304_no2, color="red", shade=True)
sns.kdeplot(data=month_231_df01_12_no2, color="blue", shade=True)
plt.show()
#등분산 X

In [ ]:
from scipy.stats import levene
levene(month_231_df01_0304_no2,month_231_df01_12_no2)
#등분산O

In [ ]:
from scipy import stats

month_no2=stats.ttest_ind(month_231_df01_0304_no2,month_231_df01_12_no2,equal_var=True,alternative="two-sided")
month_no2
# p-value > 0.05 : 벚꽃 시즌과 비벚꽃시즌에 유의미한 차이가 없음

#### o3

In [ ]:
sns.kdeplot(data=month_231_df01_0304_o3, color="red", shade=True)
sns.kdeplot(data=month_231_df01_12_o3, color="blue", shade=True)
plt.show()
#등분산 x

In [ ]:
from scipy.stats import levene
levene(month_231_df01_0304_o3,month_231_df01_12_o3)
#등분산O

In [ ]:
from scipy import stats

month_no2=stats.ttest_ind(month_231_df01_0304_o3,month_231_df01_12_o3,equal_var=True,alternative="two-sided")
month_no2
#p-value > 0.05 : 벚꽃 시즌과 비벚꽃 시즌에 유의미한 차이가 없음

### 미세먼지, 초미세먼지

In [ ]:
#결측치 보간
month_231_df02=month_231_df02.fillna(month_231_df02.mean())#no2, o3는 평균값으로
month_231_df02

In [ ]:
# 월별 그룹화
month_231_df02=month_231_df02.groupby(['month']).mean()
month_231_df02

In [ ]:
month_231_df020=month_231_df02.reset_index()
month_231_df020

In [ ]:
#월별 미세먼지 농도
sns.barplot(data = month_231_df020 , x = 'month', y = 'dust',palette = col)
plt.show()

In [ ]:
#월별 초미세먼지 농도
sns.barplot(data = month_231_df020 , x = 'month', y = 'fdust',palette = col)
plt.show()

In [ ]:
col=['#6B4E24','#EECA98','#EBAA4F','#6B5B45','#B8853E','#AB891A','#6B5610','#EBBC23','#F7B50F']
#미세먼지와 초미세먼지 월별 추세
month_231_df02[['dust','fdust']].plot(figsize=(10,6),color=col).autoscale(axis='x',tight=True)
#계절성 확인

## **3. 영등포구 요일별 분석**

In [ ]:
day=pd.read_csv('서울시 기간별 일평균 대기환경 정보_최종.csv',encoding='cp949')

In [ ]:
#칼럼이름 변경
day=day.rename(columns={'미세먼지(㎍/㎥)':'dust','이산화질소농도(ppm)':'no2','초미세먼지(㎍/㎥)':'fdust',
                        '아황산가스농도(ppm)':'so2','오존(ppm)':'o3','이산화질소농도(ppm)':'no2','일산화탄소농도(ppm)':'co'})

In [ ]:
#영등포구
day_231=day.loc[day['측정소명']=='영등포구',['측정일자','dust','fdust','no2','o3']]
day_231

In [ ]:
#측정일자 type을 string으로 변환
day_231_df=day_231.astype({ '측정일자' : 'string' })

In [ ]:
#시간정보 열을 datetime 자료형으로 변환
day_231_df['측정일자2']=pd.to_datetime(day_231_df['측정일자'], format='%Y-%m-%d') 
day_231_df

In [ ]:
#요일 칼럼 추가
day_231_df['weekday']=day_231_df['측정일자2'].dt.weekday
day_231_df

In [ ]:
#요일 이름 표기
weekday_list=['mon','tue','wed','thu','fri','sat','sun']
day_231_df['week']=day_231_df.apply(lambda x : weekday_list[x['weekday']],axis=1)
day_231_df

In [ ]:
#요일로 groupby
day_231_df0=day_231_df.groupby(by='week').mean()
day_231_df0

In [ ]:
# df1 : 이산화질소, 오존 
# df2 : 미세먼지, 초미세먼지
day_231_df1=day_231[['no2','o3']]
day_231_df2=day_231[['dust','fdust']]

### 미세먼지와 초미세먼지

In [ ]:
#미세먼지의 경우 수요일이 가장 높고, 일요일이 가장 적음
sns.barplot(data = day_231_df,x= 'week', y = 'dust',palette = col)
plt.show()

In [ ]:
#초미세먼지의 경우 수요일이 가장 높고, 토요일이 가장 적음
sns.barplot(data = day_231_df,x= 'week', y = 'fdust',palette = col)
plt.show()

In [ ]:
#이산화질소의 경우 금요일이 가장 높고, 일요일이 가장 적음
sns.barplot(data = day_231_df,x= 'week', y = 'no2',palette = col)
plt.show()

In [ ]:
#오존의 경우 일요일이 가장 높고, 수요일이 가장 적음
sns.barplot(data = day_231_df,x= 'week', y = 'o3',palette = col)
plt.show()

## **4. 영등포구 시간별 분석**

In [ ]:
#서울시 기간별 시간평균 대기환경 정보 (2019.3 ,2019.4 ,2020.3, 2020.4)
time_201903=pd.read_csv('서울시_기간별_시간평균_대기환경_정보_2019.03.csv',encoding = 'cp949')
time_201904=pd.read_csv('서울시_기간별_시간평균_대기환경_정보_2019.04.csv',encoding = 'cp949')

time_202003=pd.read_csv('서울시_기간별_시간평균_대기환경_정보_2020.03.csv',encoding = 'cp949')
time_202004=pd.read_csv('서울시_기간별_시간평균_대기환경_정보_2020.04.csv',encoding = 'cp949')

In [ ]:
time_231_202003_df1 = time_231_202003_df[((time_231_202003_df['timeline'] >= '2020-03-21 09:00') & (time_231_202003_df['timeline'] <= '2020-03-21 21:00'))]
#time_231_202003_df1.sort_values(by='dust', ascending=True, inplace=False).head()
#time_231_202003_df1.sort_values(by='fdust', ascending=True, inplace=False).head()
#time_231_202003_df1.sort_values(by='no2', ascending=True, inplace=False).head()
#time_231_202003_df1.sort_values(by='o3', ascending=True, inplace=False).head()

In [ ]:
#미세먼지 : 토요일 9시, 10시, 11시
time_231_202003_df1.sort_values(by='dust', ascending=True, inplace=False).head()

In [ ]:
#초미세먼지 : 토요일 9시, 10시, 11시
time_231_202003_df1.sort_values(by='fdust', ascending=True, inplace=False).head()

In [ ]:
# 이산화질소 : 토요일 12시, 21시, 16시
time_231_202003_df1.sort_values(by='no2', ascending=True, inplace=False).head()

In [ ]:
#오존 : 토요일 09시, 10시, 11시 
time_231_202003_df1.sort_values(by='o3', ascending=True, inplace=False).head()

### 2020년 3월 week4: sat, sun


In [ ]:
time_231_202003_df1 = time_231_202003_df[((time_231_202003_df['timeline'] >= '2020-03-28 09:00') & (time_231_202003_df['timeline'] <= '2020-03-29 21:00'))]

In [ ]:
#미세먼지 : 토요일 18시, 16시, 9시
time_231_202003_df1.sort_values(by='dust', ascending=True, inplace=False).head()

In [ ]:
#초미세먼지 : 토요일 17시, 10시, 9시
time_231_202003_df1.sort_values(by='fdust', ascending=True, inplace=False).head()

In [ ]:
#이산화질소 : 토요일 15시 ,14시, 13시
time_231_202003_df1.sort_values(by='no2', ascending=True, inplace=False).head()

In [ ]:
#오존 : 토요일 9시
time_231_202003_df1.sort_values(by='o3', ascending=True, inplace=False).head(7)

### 2020년 4월 week1: sat, sun

In [ ]:
time_231_202004_df1 = time_231_202004_df[((time_231_202004_df['timeline'] >= '2020-04-04 09:00') & (time_231_202004_df['timeline'] <= '2020-04-05 21:00'))]

In [ ]:
#미세먼지 : 토요일 21시, 20시, 일요일 20시
time_231_202004_df1.sort_values(by='dust', ascending=True, inplace=False).head(15)

In [ ]:
#초미세먼지 : 토요일 13시, 20시, 15시
time_231_202004_df1.sort_values(by='fdust', ascending=True, inplace=False).head(7)

In [ ]:
#이산화질소 : 일요일 16시, 토요일 14시, 16시
time_231_202004_df1.sort_values(by='no2', ascending=True, inplace=False).head()

In [ ]:
#오존 : 토요일 9시
time_231_202004_df1.sort_values(by='o3', ascending=True, inplace=False).head(9)

### 2020년 4월 week2: sat, sun

In [ ]:
time_231_202004_df1 = time_231_202004_df[((time_231_202004_df['timeline'] >= '2020-04-11 09:00') & (time_231_202004_df['timeline'] <= '2020-04-12 21:00'))]

In [ ]:
#미세먼지 : 일요일 20시, 17시, 18시
time_231_202004_df1.sort_values(by='dust', ascending=True, inplace=False).head()

In [ ]:
#초미세먼지 : 일요일 21시, 19시, 18시
time_231_202004_df1.sort_values(by='fdust', ascending=True, inplace=False).head()

In [ ]:
#초미세먼지 : 일요일 17시, 16시, 15시
time_231_202004_df1.sort_values(by='no2', ascending=True, inplace=False).head()

In [ ]:
#오존 
time_231_202004_df1.sort_values(by='o3', ascending=True, inplace=False).head(10)

# 7. 송파구 <a class="anchor" id="7.송파구"></a>
---
### 9기 김서진

## 데이터 로딩 및 설정

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm

In [ ]:
# 설치된 폰트 확인
sys_font = fm.findSystemFonts()
print("sys_font number : {}".format(len(sys_font)))
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]

In [ ]:
# 나눔 폰트 업로드
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
# 폰트가 정상적으로 설치됨.
sys_font = fm.findSystemFonts()
print("sys_font number : {}".format(len(sys_font)))

nanum_font = [f for f in sys_font if 'Nanum' in f]
print("nanum_font number : {}".format(len(nanum_font)))

In [ ]:
# 현재 설정되어 있는 폰트 사이즈와 글꼴 알아보기
def current_font():
  print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능
        
current_font()

In [ ]:
path = '/Users/gimseojin/Library/Fonts/NanumGothic.ttf' # 원하는 폰트의 경로 가져오기
font_name = fm.FontProperties(fname = path, size = 10).get_name()
print(font_name)
plt.rc('font', family = font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False # 그래프의 x, y축에 정상적인 - 표시 설정

# 런타임 다시 시작!

In [ ]:
# 이거 한 줄 적어주시면 title 한글로 써도 안 깨질거에요!
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
col=['#6B4E24','#EECA98','#EBAA4F','#6B5B45','#B8853E','#AB891A','#6B5610','#EBBC23','#F7B50F']

In [ ]:
# data loading
#time_df = pd.read_csv('/content/EDA_data_new/서울시 기간별 시간평균 대기환경 정보.csv', encoding = 'EUC-KR', skiprows=0)
day_df = pd.read_csv('/content/EDA_data_new/서울시 기간별 일평균 대기환경 정보.csv', encoding = 'EUC-KR', skiprows=0)
month_df = pd.read_csv('/content/EDA_data_new/서울시 월별 평균 대기오염도 정보.csv', encoding = 'EUC-KR', skiprows=0)

In [ ]:
# 2019 time data
sp_time_df = pd.read_csv('songpa_2019.csv')

In [ ]:
sp_time_df.head()

## 1. 시간대별 데이터 분석

In [ ]:
where = time_df.groupby('측정소명')

In [ ]:
where.mean()

In [ ]:
sp_time_df = time_df[time_df['측정소명']=='송파구']
sp_time_df = sp_time_df[['측정일시', '미세먼지 1시간(㎍/㎥)',
       '미세먼지 24시간(㎍/㎥)', '초미세먼지(㎍/㎥)', '오존(ppm)', '이산화질소농도(ppm)']]
sp_time_df.head()

In [ ]:
from datetime import datetime

datetime_object = sp_time_df['측정일시'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

print(type(datetime_object))
print(datetime_object)  # printed in default format

In [ ]:
sp_time_df['Time'] = datetime_object
sp_time_df.head()

In [ ]:
sp_day = sp_time_df[sp_time_df['Hour']>=9]
sp_day = sp_day[sp_day['Hour']<22]

In [ ]:
sp_night = sp_time_df[sp_time_df['Hour']<9]
sp_night = sp_night.append(sp_time_df[sp_time_df['Hour']>=22])

In [ ]:
sp_day.describe()

In [ ]:
sp_night.describe()

In [ ]:
plt.rcParams['figure.figsize'] = (9, 8)

In [ ]:
plt.subplot(2,2,1)
plt.barh(np.arange(2), [sp_day['Dust'].mean(), sp_night['Dust'].mean()], color=col, linewidth=3)
plt.yticks(np.arange(2), ['Day','Night'])
plt.title('오전 오후 미세먼지 농도 비교')

plt.subplot(2,2,2)
plt.barh(np.arange(2), [sp_day['F_dust'].mean(), sp_night['F_dust'].mean()], color=col, linewidth=3)
plt.yticks(np.arange(2), ['Day','Night'])
plt.title('오전 오후 초미세먼지 농도 비교')

plt.subplot(2,2,3)
plt.barh(np.arange(2), [sp_day['NO2'].mean(), sp_night['NO2'].mean()], color=col, linewidth=3)
plt.yticks(np.arange(2), ['Day','Night'])
plt.title('오전 오후 이산화질소 농도 비교')

plt.subplot(2,2,4)
plt.barh(np.arange(2), [sp_day['O3'].mean(), sp_night['O3'].mean()], color=col, linewidth=3)
plt.yticks(np.arange(2), ['Day','Night'])
plt.title('오전 오후 오존 농도 비교')

plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(2,2,1)
plt.boxplot([sp_day['Dust'],sp_night['Dust']], whis=2)
plt.xticks(np.arange(3), ['','Day','Night'])
plt.title('Dust')

plt.subplot(2,2,2)
plt.boxplot([sp_day['F_dust'],sp_night['F_dust']], whis=2)
plt.xticks(np.arange(3), ['','Day','Night'])
plt.title('F_dust')

plt.subplot(2,2,3)
plt.boxplot([sp_day['NO2'],sp_night['NO2']], whis=2)
plt.xticks(np.arange(3), ['','Day','Night'])
plt.title('NO2')

plt.subplot(2,2,4)
plt.boxplot([sp_day['O3'],sp_night['O3']], whis=2)
plt.xticks(np.arange(3), ['','Day','Night'])
plt.title('O3')

plt.tight_layout()
plt.show()


In [ ]:
time_group = sp_time_df.groupby('Hour')
mean_time = time_group.mean()

In [ ]:
plt.subplot(2,2,1)
plt.plot(mean_time.index, mean_time['Dust'], color=col[0])
plt.title('시간대별 미세먼지 농도 변화 추이')

plt.subplot(2,2,2)
plt.plot(mean_time.index, mean_time['NO2'], color=col[1])
plt.title('시간대별 이산화질소 농도 변화 추이')

plt.subplot(2,2,3)
plt.plot(mean_time.index, mean_time['F_dust'], color=col[2])
plt.title('시간대별 초미세먼지 농도 변화 추이')

plt.subplot(2,2,4)
plt.plot(mean_time.index, mean_time['O3'], color=col[3])
plt.title('시간대별 오존 농도 변화 추이')

plt.tight_layout()
plt.show()

미세먼지, 오존의 경우 활동 시간대에 높아지고 이산화질소의 경우 반대로 활동 시간대에 낮아지고 출퇴근 시간대에 높아지는 특성을 보인다. 이는 자동차 배기가스로 대부분의 이산화질소가 발생하기에 생겨나는 일인 것으로 생각된다. 
+) 교통량 데이터와의 결합 분석 진행하기

In [ ]:
sp_time_df.head()

In [ ]:
### 의현님 코드 올려주신 것 보고 적용해보면 좋을 것 같아서 따라서 해봤습니다!
for (i, row) in sp_time_df.iterrows(): 
    dust = row[1] 
    k = '오류' 
     
    if dust < 31: 
        k = '좋음' 
    elif dust < 81: 
        k = '보통' 
    elif dust < 151: 
        k = '나쁨' 
    else: 
        k = '매우 나쁨' 
    sp_time_df.loc[i, 'dust_alarm'] = k
# 초미세먼지 농도가  0 ~ 15은 0, 16 ~ 35은 1, 36 ~ 75은 2, 76 이상은 3으로 등급을 나눈다 
 
for (i, row) in sp_time_df.iterrows(): 
    f_dust = row[2] 
    k = '오류' 
     
    if f_dust < 16: 
        k = '좋음' 
    elif f_dust < 36: 
        k = '보통' 
    elif f_dust < 76: 
        k = '나쁨' 
    else: 
        k = '매우 나쁨' 
    sp_time_df.loc[i, 'f_dust_alarm'] = k

In [ ]:
dc = sp_time_df['dust_alarm'].value_counts() 
print(dc) 
 
fig, axes = plt.subplots(2, 1, figsize=(8,12)) 
 
sns.barplot(ax = axes[0], x = dc.index, 
            y = dc.values, palette = col).set_title('연간 미세먼지 경보 뱔령 횟수')
axes[1].pie(labels = dc.index, x = dc.values, 
            explode = [0.03, 0.03, 0.1, 0.1], colors = col, 
            textprops = {'fontsize' : 12}, autopct = "%.2f%%") 
plt.title('연간 송파구 미세먼지 등급 발령 횟수 (1시간)', size = 16) 
plt.subplots_adjust(left = 0.125, bottom = 0.1,  right = 0.9, top = 0.9, wspace = 0.2)
plt.show()

In [ ]:
week = ['월', '화', '수', '목', '금', '토', '일'] 
 
sp_time_df['요일'] = sp_time_df.apply(lambda row: week[row['Date'].weekday()], axis = 1)

df_al = sp_time_df[sp_time_df['Dust'] >= 81] 
days = df_al.groupby(['요일']).agg({'dust_alarm' : 'count'}).reset_index() 
days


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8,12)) 
 
sns.barplot(ax = axes[0], x = days['요일'], 
            y = days['dust_alarm'], palette = col).set_title('요일별 송파구 미세먼지 등급 발령 횟수')
#axes[0].axis([-0.5, 6.5, 80, 200]) 
axes[1].pie(labels = days['요일'], x = days['dust_alarm'], 
            colors = col, textprops = {'fontsize' : 12}, autopct = "%.2f%%") 
plt.title('요일별 송파구 미세먼지 등급 발령', size = 16) 
plt.subplots_adjust(left = 0.125, bottom = 0.1,  right = 0.9, top = 0.9, wspace = 0.2)
plt.show()

In [ ]:
sp_time_df['Month'] = datetime_object.apply(lambda x: x.month)
time_34 = sp_time_df[sp_time_df['Month']==3]
time_34 = time_34.append(sp_time_df[sp_time_df['Month']==4])
time_34

In [ ]:
group_time = time_34.groupby('Hour').mean()

In [ ]:
plt.subplot(2,2,1)
group_time.sort_values('Dust',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['Dust'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 미세먼지 농도')

plt.subplot(2,2,2)
group_time.sort_values('F_dust',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['F_dust'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 초미세먼지 농도')

plt.subplot(2,2,3)
group_time.sort_values('NO2',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['NO2'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 이산화질소 농도')

plt.subplot(2,2,4)
group_time.sort_values('O3',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['O3'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 오존 농도')

plt.tight_layout()
plt.show

In [ ]:
group_time.sort_values('F_dust',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['F_dust'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 초미세먼지 농도')

In [ ]:
group_time.sort_values('NO2',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['NO2'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 이산화질소 농도')

In [ ]:
group_time.sort_values('O3',ascending=True, inplace=True)
plt.barh(np.arange(24),group_time['O3'],color=col)
plt.yticks(np.arange(24),group_time.index)
plt.title('송파구 시간대별 오존 농도')

In [ ]:
group_days = time_34.groupby('요일').mean()

plt.subplot(2,2,1)
group_days.sort_values('Dust',ascending=True, inplace=True)
plt.barh(np.arange(7),group_days['Dust'],color=col)
plt.yticks(np.arange(7),group_days.index)
plt.title('송파구 요일별 미세먼지 농도')

plt.subplot(2,2,2)
group_days.sort_values('F_dust',ascending=True, inplace=True)
plt.barh(np.arange(7),group_days['F_dust'],color=col)
plt.yticks(np.arange(7),group_days.index)
plt.title('송파구 요일별 초미세먼지 농도')

plt.subplot(2,2,3)
group_days.sort_values('NO2',ascending=True, inplace=True)
plt.barh(np.arange(7),group_days['NO2'],color=col)
plt.yticks(np.arange(7),group_days.index)
plt.title('송파구 요일별 이산화질소 농도')

plt.subplot(2,2,4)
group_days.sort_values('O3',ascending=True, inplace=True)
plt.barh(np.arange(7),group_days['O3'],color=col)
plt.yticks(np.arange(7),group_days.index)
plt.title('송파구 요일별 오존 농도')

plt.tight_layout()
plt.show()

In [ ]:
group_time.drop('Month', axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sp_scaled = pd.DataFrame(sc.fit_transform(group_time), index = group_time.index, columns = group_time.columns)
sp_scaled

In [ ]:
sp_scaled['Sum'] = sp_scaled['Dust']+sp_scaled['F_dust']+sp_scaled['NO2']+sp_scaled['O3']

In [ ]:
sp_scaled.sort_values('Sum', inplace=True)

In [ ]:
sp_scaled_10 = sp_scaled['Sum'][:10].apply(lambda x: -x)
sp_scaled_10.sort_values(ascending=True, inplace=True)

In [ ]:
plt.barh(np.arange(10),sp_scaled_10, color=col)
plt.yticks(np.arange(10),sp_scaled_10.index)
plt.title('송파구 점수 합산 TOP 10')

In [ ]:
def week_no(d):
    target_day = d # 4
    firstday = target_day.replace(day=1) # 5
    while firstday.weekday() != 0: # 6
        firstday += timedelta(days=1)
      
    if target_day < firstday: # 7
        return 0
  
    return (target_day - firstday).days // 7 + 1 # 8

In [ ]:
time_34['Week_no'] = time_34['Date'].apply(lambda x: week_no(x))
time_34

In [ ]:
time_3 = time_34[time_34['Month']==3]
time_4 = time_34[time_34['Month']==4]
time_3['Week_no'] +=1
time_3

In [ ]:
time_3_3 = time_3[time_3['Week_no']==3]
time_3_4 = time_3[time_3['Week_no']==4]
time_3_5 = time_3[time_3['Week_no']==5]
time_4_1 = time_4[time_4['Week_no']==1]
time_4_2 = time_4[time_4['Week_no']==2]

In [ ]:
group_time33 = time_3_3.groupby('Hour').mean()
group_time34 = time_3_4.groupby('Hour').mean()
group_time35 = time_3_5.groupby('Hour').mean()
group_time41 = time_4_1.groupby('Hour').mean()
group_time42 = time_4_2.groupby('Hour').mean()

In [ ]:
group_time33.drop('Month', axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler

sc33 = StandardScaler()
sp_scaled33 = pd.DataFrame(sc33.fit_transform(group_time33), index = group_time33.index, columns = group_time33.columns)
sp_scaled33['Sum'] = sp_scaled33['Dust']+sp_scaled33['F_dust']+sp_scaled33['NO2']+sp_scaled33['O3']
sp_scaled33.sort_values('Sum', inplace=True)
sp_scaled33_10 = sp_scaled33['Sum'][:10].apply(lambda x: -x)
sp_scaled33_10.sort_values(ascending=True, inplace=True)
plt.barh(np.arange(10),sp_scaled33_10, color=col)
plt.yticks(np.arange(10),sp_scaled33_10.index)
plt.title('송파구 3월 3주차 점수 합산 TOP 10')

In [ ]:
#group_time34.drop('Month', axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler

sc34 = StandardScaler()
sp_scaled34 = pd.DataFrame(sc34.fit_transform(group_time34), index = group_time34.index, columns = group_time34.columns)
sp_scaled34['Sum'] = sp_scaled34['Dust']+sp_scaled34['F_dust']+sp_scaled34['NO2']+sp_scaled34['O3']
sp_scaled34.sort_values('Sum', inplace=True)
sp_scaled34_10 = sp_scaled34['Sum'][:10].apply(lambda x: -x)
sp_scaled34_10.sort_values(ascending=True, inplace=True)
plt.barh(np.arange(10),sp_scaled34_10, color=col)
plt.yticks(np.arange(10),sp_scaled34_10.index)
plt.title('송파구 3월 4주차 점수 합산 TOP 10')

In [ ]:
group_time35.drop('Month', axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler

sc35 = StandardScaler()
sp_scaled35 = pd.DataFrame(sc35.fit_transform(group_time35), index = group_time35.index, columns = group_time35.columns)
sp_scaled35['Sum'] = sp_scaled35['Dust']+sp_scaled35['F_dust']+sp_scaled35['NO2']+sp_scaled35['O3']
sp_scaled35.sort_values('Sum', inplace=True)
sp_scaled35_10 = sp_scaled35['Sum'][:10].apply(lambda x: -x)
sp_scaled35_10.sort_values(ascending=True, inplace=True)
plt.barh(np.arange(10),sp_scaled35_10, color=col)
plt.yticks(np.arange(10),sp_scaled35_10.index)
plt.title('송파구 3월 5주차 점수 합산 TOP 10')

In [ ]:
group_time41.drop('Month', axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler

sc41 = StandardScaler()
sp_scaled41 = pd.DataFrame(sc41.fit_transform(group_time41), index = group_time41.index, columns = group_time41.columns)
sp_scaled41['Sum'] = sp_scaled41['Dust']+sp_scaled41['F_dust']+sp_scaled41['NO2']+sp_scaled41['O3']
sp_scaled41.sort_values('Sum', inplace=True)
sp_scaled41_10 = sp_scaled41['Sum'][:10].apply(lambda x: -x)
sp_scaled41_10.sort_values(ascending=True, inplace=True)
plt.barh(np.arange(10),sp_scaled41_10, color=col)
plt.yticks(np.arange(10),sp_scaled41_10.index)
plt.title('송파구 4월 1주차 점수 합산 TOP 10')

In [ ]:
group_time42.drop('Month', axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler

sc42 = StandardScaler()
sp_scaled42 = pd.DataFrame(sc42.fit_transform(group_time42), index = group_time42.index, columns = group_time42.columns)
sp_scaled42['Sum'] = sp_scaled42['Dust']+sp_scaled42['F_dust']+sp_scaled42['NO2']+sp_scaled42['O3']
sp_scaled42.sort_values('Sum', inplace=True)
sp_scaled42_10 = sp_scaled42['Sum'][:10].apply(lambda x: -x)
sp_scaled42_10.sort_values(ascending=True, inplace=True)
plt.barh(np.arange(10),sp_scaled42_10, color=col)
plt.yticks(np.arange(10),sp_scaled42_10.index)
plt.title('송파구 4월 2주차 점수 합산 TOP 10')

## 2. 일별 데이터 분석 _ 요일별 특성 파악

In [ ]:
sp_day_df = day_df[day_df['측정소명']=='송파구']
sp_day_df = sp_day_df[['측정일자', '미세먼지(㎍/㎥)','초미세먼지(㎍/㎥)', '오존(ppm)', '이산화질소농도(ppm)']]
sp_day_df.head()

In [ ]:
sp_day_df.rename(columns={"이산화질소농도(ppm)":"NO2", "오존(ppm)":"O3",
                            "미세먼지(㎍/㎥)":"Dust", "초미세먼지(㎍/㎥)":"F_dust"}, inplace=True)
sp_day_df.head()

In [ ]:
sp_day_df['Day'] = sp_day_df['측정일자'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d'))

In [ ]:
day_list = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
sp_day_df['Weekday'] = sp_day_df['Day'].apply(lambda x: day_list[x.weekday()])

In [ ]:
weekday_grp = sp_day_df.groupby('Weekday')

In [ ]:
day_mean = weekday_grp.mean()
day_mean = day_mean.reindex(day_list, axis='index')
day_mean

In [ ]:
plt.subplot(1,2,1)
plt.bar(np.arange(7), day_mean['Dust'], color = col)
plt.xticks(np.arange(7), day_mean.index)
plt.title('송파구 요일별 미세먼지 추이')

plt.subplot(1,2,2)
plt.bar(np.arange(7), day_mean['F_dust'], color = col)
plt.xticks(np.arange(7), day_mean.index)
plt.title('송파구 요일별 초미세먼지 추이')

plt.tight_layout()
plt.show()

## 3. 벚꽃 개화시기 분석

In [ ]:
month_group = sp_month_df.groupby('Month')
mean_month = month_group.mean()
mean_month.drop('측정월', axis = 1, inplace = True)
mean_month

In [ ]:
plt.subplot(2,1,1)
plt.bar(np.arange(12), mean_month['Dust'], color = col)
plt.xticks(np.arange(12), mean_month.index)
plt.title('송파구 월별 미세먼지 추이')

plt.subplot(2,1,2)
plt.bar(np.arange(12), mean_month['F_dust'], color = col)
plt.xticks(np.arange(12), mean_month.index)
plt.title('송파구 월별 초미세먼지 추이')

plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(2,1,1)
plt.bar(np.arange(12), mean_month['NO2'], color = col)
plt.xticks(np.arange(12), mean_month.index)
plt.title('송파구 월별 이산화질소 추이')

plt.subplot(2,1,2)
plt.bar(np.arange(12), mean_month['O3'], color = col)
plt.xticks(np.arange(12), mean_month.index)
plt.title('송파구 월별 오존 추이')

plt.tight_layout()
plt.show()

In [ ]:
fig1 = plt.subplot(1,2,1)
plt.bar(np.arange(2), data[['Dust','F_dust']], color = col)
plt.xticks(np.arange(2), data[['Dust','F_dust']].index)
plt.title('벚꽃시즌 (초)미세먼지 평균 농도')

fig2 = plt.subplot(1,2,2, sharey=fig1)
plt.bar(np.arange(2), data2[['Dust','F_dust']], color = col)
plt.xticks(np.arange(2), data2[['Dust','F_dust']].index)
plt.title('비벚꽃시즌 (초)미세먼지 평균 농도')

plt.tight_layout()
plt.show()

In [ ]:
fig1 = plt.subplot(1,2,1)
plt.bar(np.arange(2), data[['NO2','O3']], color = col)
plt.xticks(np.arange(2), data[['NO2','O3']].index)
plt.title('벚꽃시즌 이산화질소/오존 평균 농도')

fig2 = plt.subplot(1,2,2, sharey=fig1)
plt.bar(np.arange(2), data2[['NO2','O3']], color = col)
plt.xticks(np.arange(2), data2[['NO2','O3']].index)
plt.title('비벚꽃시즌 이산화질소/오존 평균 농도')

plt.tight_layout()
plt.show()

In [ ]:
df_month = sp_month_df[sp_month_df['Dust'] >= 81] 
df_mon = df_month.groupby(['Month']).agg({'Dust' : 'count'}).reset_index() 
df_mon

In [ ]:
plt.figure(figsize = (8, 6)) 
 
sns.barplot(x = df_mon['Month'], y = df_mon['Dust'], 
            palette = col) 
plt.title('미세먼지 경보 발령 횟수 (월)', fontsize = 20) 
plt.show()